Step 1: Model Interpretability & Explainability

In [34]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

def realistic_mlp_predictor(customer_data):
    """Use YOUR trained MLP model with realistic business logic adjustments"""
    
    print("🤖 Loading YOUR trained MLP model...")
    
    # Load YOUR trained model and transformers
    model = load_model('models/credit_score_mlp_model.h5')
    quantile_transformer = joblib.load('models/quantile_transformer.pkl')
    poly_features = joblib.load('models/poly_features.pkl')
    final_scaler = joblib.load('models/final_scaler.pkl')
    top_feature_indices = joblib.load('models/top_feature_indices.pkl')
    feature_names = joblib.load('models/feature_names.pkl')
    
    print("🧠 Using your neural network for prediction...")
    
    # Convert customer data to DataFrame
    df_input = pd.DataFrame([customer_data])
    
    # Ensure all required features are present
    for feature in feature_names:
        if feature not in df_input.columns:
            df_input[feature] = 0
    
    # Reorder columns to match training data
    df_input = df_input[feature_names]
    
    # Apply YOUR preprocessing pipeline
    X_quantile = quantile_transformer.transform(df_input.values)
    X_top = df_input.values[:, top_feature_indices]
    X_poly = poly_features.transform(X_top)
    X_enhanced = np.concatenate([X_quantile, X_poly], axis=1)
    X_final = final_scaler.transform(X_enhanced)
    
    # Get prediction from YOUR MLP MODEL
    raw_prediction = model.predict(X_final, verbose=0)[0][0]
    
    print(f"🎯 Your MLP model predicts: {raw_prediction:.0f}")
    
    # Apply realistic business adjustments to YOUR model's output
    income = customer_data.get('INCOME', 0)
    debt = customer_data.get('DEBT', 0)
    savings = customer_data.get('SAVINGS', 0)
    
    # Calculate financial reality
    net_worth = savings - debt
    debt_coverage = (savings / debt) if debt > 0 else float('inf')
    
    # Adjust YOUR model's prediction based on financial reality
    adjusted_score = raw_prediction
    adjustments = []
    
    # If savings can cover debt completely
    if debt_coverage >= 1.0 and net_worth >= 0:
        boost = min(100, 50 + (net_worth / income * 20)) if income > 0 else 50
        adjusted_score += boost
        adjustments.append(f"Debt fully covered by savings: +{boost:.0f}")
    
    # High net worth bonus
    if income > 0:
        net_worth_ratio = net_worth / income
        if net_worth_ratio > 3:
            boost = min(75, net_worth_ratio * 15)
            adjusted_score += boost
            adjustments.append(f"Excellent net worth: +{boost:.0f}")
    
    # Cap the final score
    final_score = min(850, max(300, adjusted_score))
    
    # Determine approval based on final score
    if final_score >= 740:
        approval = "Excellent"
        apr = "3-7%"
    elif final_score >= 670:
        approval = "High"
        apr = "6-12%"
    elif final_score >= 580:
        approval = "Medium"
        apr = "10-18%"
    else:
        approval = "Low"
        apr = "15-25%"
    
    return {
        'raw_mlp_score': round(raw_prediction, 0),
        'final_score': round(final_score, 0),
        'adjustments': adjustments,
        'approval': approval,
        'apr': apr,
        'model_used': 'Your Trained MLP + Business Logic'
    }

def get_user_input():
    """Get financial information from user"""
    
    print("🏦 Credit Score Calculator (Using YOUR MLP Model)")
    print("Please enter your financial information:")
    print("-" * 50)
    
    # Get income
    while True:
        try:
            income = float(input("Enter your annual income (₹): "))
            if income < 0:
                print("Income cannot be negative. Please try again.")
                continue
            break
        except ValueError:
            print("Please enter a valid number.")
    
    # Get debt
    while True:
        try:
            debt = float(input("Enter your total debt (₹): "))
            if debt < 0:
                print("Debt cannot be negative. Please try again.")
                continue
            break
        except ValueError:
            print("Please enter a valid number.")
    
    # Get savings
    while True:
        try:
            savings = float(input("Enter your total savings (₹): "))
            if savings < 0:
                print("Savings cannot be negative. Please try again.")
                continue
            break
        except ValueError:
            print("Please enter a valid number.")
    
    return {
        'INCOME': income,
        'DEBT': debt,
        'SAVINGS': savings
    }

def main():
    """Main function using YOUR MLP model"""
    
    # Get user input
    customer = get_user_input()
    
    # Use YOUR MLP model for prediction
    result = realistic_mlp_predictor(customer)
    
    # Display results
    print("\n" + "=" * 50)
    print("🧠 YOUR MLP MODEL RESULTS")
    print("=" * 50)
    
    print(f"Income: ₹{customer['INCOME']:,.0f}")
    print(f"Debt: ₹{customer['DEBT']:,.0f}")
    print(f"Savings: ₹{customer['SAVINGS']:,.0f}")
    print()
    print(f"🤖 Raw MLP Prediction: {result['raw_mlp_score']}")
    print(f"📊 Final Credit Score: {result['final_score']}")
    print(f"💳 Loan Approval: {result['approval']}")
    print(f"💰 Expected APR: {result['apr']}")
    
    if result['adjustments']:
        print(f"\n🔧 Business Logic Adjustments:")
        for adj in result['adjustments']:
            print(f"   • {adj}")
    
    print(f"\n🎯 Model Used: {result['model_used']}")
    print("=" * 50)
    
    # Ask if user wants to calculate again
    while True:
        again = input("\nCalculate for another person? (y/n): ").lower()
        if again in ['y', 'yes']:
            print("\n" + "-" * 50)
            main()
            break
        elif again in ['n', 'no']:
            print("Thank you for using YOUR MLP Credit Calculator! 🤖")
            break
        else:
            print("Please enter 'y' for yes or 'n' for no.")

# Run YOUR MLP-based calculator
if __name__ == "__main__":
    main()


🏦 Credit Score Calculator (Using YOUR MLP Model)
Please enter your financial information:
--------------------------------------------------


🤖 Loading YOUR trained MLP model...
🧠 Using your neural network for prediction...
🎯 Your MLP model predicts: 563

🧠 YOUR MLP MODEL RESULTS
Income: ₹1,000,000
Debt: ₹500,000
Savings: ₹500,000

🤖 Raw MLP Prediction: 563.0
📊 Final Credit Score: 613.0
💳 Loan Approval: Medium
💰 Expected APR: 10-18%

🔧 Business Logic Adjustments:
   • Debt fully covered by savings: +50

🎯 Model Used: Your Trained MLP + Business Logic
Thank you for using YOUR MLP Credit Calculator! 🤖
